In [290]:
import numpy as np
from evidently.metrics import EmbeddingsDriftMetric, Comment
from evidently.report import Report
from evidently.metrics.data_drift.embedding_drift_methods import model, ratio, mmd
import pickle
import pandas as pd
from evidently import ColumnMapping
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
from tensorflow.keras.models import load_model

In [291]:
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

In [292]:
from deepchecks.vision import classification_dataset_from_directory
import json
import matplotlib.pyplot as plt
from deepchecks.vision.suites import full_suite
from deepchecks.vision.suites import full_suite

In [293]:
client = MlflowClient()

In [294]:
#set experiment
mlflow.set_experiment('Drift Analysis using normalized embeddings extracted from autoencoder v3')

<Experiment: artifact_location='file:///Users/ximenamoure/Desktop/drift_last/mlruns/324168497884912629', creation_time=1726150198924, experiment_id='324168497884912629', last_update_time=1726150198924, lifecycle_stage='active', name='Drift Analysis using normalized embeddings extracted from autoencoder v3', tags={}>

In [295]:
reference_split_name = "SPLIT5(ref)"
current_split_name = "SPLIT5_transformed_10_aut"
file_name = 'embs_normalized_drift_s5_vs_s5_transf_10_enc.html'
full_path = '/Users/ximenamoure/Desktop/drift_last/drift_reports/autoencoder_v3'

In [296]:
""" # Deepchecks 
dataset_path1 = '/Users/ximenamoure/Desktop/drift_last/reference_dataset'
dataset_path2 = '/Users/ximenamoure/Chess-Piece-Classification-Dataset/images/processed/occupancy/split4'
# Save the suite results as an HTML file with detailed information
html_path = 'drift_check_results_s5_vs_s4_deepchecks.html'
train_ds_name = "Split5"
test_ds_name = "Split4" """

' # Deepchecks \ndataset_path1 = \'/Users/ximenamoure/Desktop/drift_last/reference_dataset\'\ndataset_path2 = \'/Users/ximenamoure/Chess-Piece-Classification-Dataset/images/processed/occupancy/split4\'\n# Save the suite results as an HTML file with detailed information\nhtml_path = \'drift_check_results_s5_vs_s4_deepchecks.html\'\ntrain_ds_name = "Split5"\ntest_ds_name = "Split4" '

In [297]:
# Load embeddings from .pkl files
with open("/Users/ximenamoure/Desktop/drift_last/embeddings_autoencoder_v3/x_ref_embeddings_autov3.pkl", "rb") as f:
    embeddings_train = pickle.load(f)


In [298]:
with open("/Users/ximenamoure/Desktop/drift_last/embeddings_autoencoder_v3/ref_embeddings_autov3_10_trans.pkl", "rb") as f:
    embeddings_test = pickle.load(f)

In [299]:
""" # Load embeddings from .npy files
embeddings_train = np.load("/Users/ximenamoure/Desktop/drift_last/production_model_features_last/reference/reference_features.npy")
embeddings_test = np.load("/Users/ximenamoure/Desktop/drift_last/production_model_features_last/ref_features_prod_model_20tra_.npy") """

' # Load embeddings from .npy files\nembeddings_train = np.load("/Users/ximenamoure/Desktop/drift_last/production_model_features_last/reference/reference_features.npy")\nembeddings_test = np.load("/Users/ximenamoure/Desktop/drift_last/production_model_features_last/ref_features_prod_model_20tra_.npy") '

In [300]:
if not isinstance(embeddings_train, np.ndarray):
    embeddings_train = np.array(embeddings_train)

if not isinstance(embeddings_test, np.ndarray):
    embeddings_test = np.array(embeddings_test)

In [301]:
embeddings_test

array([[ -1.7055386 ,  -3.1910574 ,   5.840725  , ...,   7.177822  ,
         -2.860446  ,  -8.658086  ],
       [  3.1875327 ,   7.2673125 ,  10.117247  , ...,  20.731285  ,
          0.6189734 ,  -8.819734  ],
       [  6.7730303 ,   0.2008654 ,  -3.237001  , ...,  -5.3264484 ,
         -8.221491  ,  10.008629  ],
       ...,
       [ -1.3509235 ,  -0.03419232,   3.2369564 , ...,  -4.6609635 ,
         -4.1752343 ,  13.211972  ],
       [  0.45131055,   2.4122355 ,   1.7833797 , ...,   1.1689043 ,
          4.714209  ,  -4.679858  ],
       [  1.6744896 ,   2.9815855 ,  -1.1983256 , ..., -11.513319  ,
         -9.4208355 ,  11.057816  ]], dtype=float32)

In [302]:
embeddings_train

array([[ -1.7055386 ,  -3.1910574 ,   5.840725  , ...,   7.177822  ,
         -2.860446  ,  -8.658086  ],
       [  3.1875327 ,   7.2673125 ,  10.117247  , ...,  20.731285  ,
          0.6189734 ,  -8.819734  ],
       [  6.7730303 ,   0.2008654 ,  -3.237001  , ...,  -5.3264484 ,
         -8.221491  ,  10.008629  ],
       ...,
       [ -1.3509235 ,  -0.03419232,   3.2369564 , ...,  -4.6609635 ,
         -4.1752343 ,  13.211972  ],
       [  0.45131055,   2.4122355 ,   1.7833797 , ...,   1.1689043 ,
          4.714209  ,  -4.679858  ],
       [  1.6744896 ,   2.9815855 ,  -1.1983256 , ..., -11.513319  ,
         -9.4208355 ,  11.057816  ]], dtype=float32)

In [303]:
len(embeddings_test)

48317

In [304]:
# Check if the arrays are the same
are_equal = np.array_equal(embeddings_train, embeddings_test)
print("Are the embeddings the same?", are_equal)

Are the embeddings the same? False


In [305]:
# Z-Score Normalization
scaler_z = StandardScaler()
embeddings_train_z = scaler_z.fit_transform(embeddings_train)
embeddings_test_z = scaler_z.transform(embeddings_test)

In [306]:
# Min-Max Normalization
scaler_minmax = MinMaxScaler()
embeddings_train_minmax = scaler_minmax.fit_transform(embeddings_train)
embeddings_test_minmax = scaler_minmax.transform(embeddings_test)

In [307]:
# L2 Normalization
embeddings_train_l2 = normalize(embeddings_train, norm='l2')
embeddings_test_l2 = normalize(embeddings_test, norm='l2')

In [308]:
# Convert to DataFrames
embeddings_train_z_df = pd.DataFrame(embeddings_train_z, columns=[f"z_{i}" for i in range(embeddings_train_z.shape[1])])
embeddings_test_z_df = pd.DataFrame(embeddings_test_z, columns=[f"z_{i}" for i in range(embeddings_test_z.shape[1])])

embeddings_train_minmax_df = pd.DataFrame(embeddings_train_minmax, columns=[f"minmax_{i}" for i in range(embeddings_train_minmax.shape[1])])
embeddings_test_minmax_df = pd.DataFrame(embeddings_test_minmax, columns=[f"minmax_{i}" for i in range(embeddings_test_minmax.shape[1])])

embeddings_train_l2_df = pd.DataFrame(embeddings_train_l2, columns=[f"l2_{i}" for i in range(embeddings_train_l2.shape[1])])
embeddings_test_l2_df = pd.DataFrame(embeddings_test_l2, columns=[f"l2_{i}" for i in range(embeddings_test_l2.shape[1])])

In [309]:
# Combine all the normalized embeddings into a single DataFrame
embeddings_train_combined_df = pd.concat([embeddings_train_z_df, embeddings_train_minmax_df, embeddings_train_l2_df], axis=1)
embeddings_test_combined_df = pd.concat([embeddings_test_z_df, embeddings_test_minmax_df, embeddings_test_l2_df], axis=1)


In [310]:
print(embeddings_train_combined_df.columns)


Index(['z_0', 'z_1', 'z_2', 'z_3', 'z_4', 'z_5', 'z_6', 'z_7', 'z_8', 'z_9',
       ...
       'l2_340', 'l2_341', 'l2_342', 'l2_343', 'l2_344', 'l2_345', 'l2_346',
       'l2_347', 'l2_348', 'l2_349'],
      dtype='object', length=1050)


In [311]:
# Define the column mapping based on the different normalization methods
column_mapping = ColumnMapping(
    embeddings={
        'embeddings_z': [f"z_{i}" for i in range(embeddings_train_z_df.shape[1])],
        'embeddings_minmax': [f"minmax_{i}" for i in range(embeddings_train_minmax_df.shape[1])],
        'embeddings_l2': [f"l2_{i}" for i in range(embeddings_train_l2_df.shape[1])]
    }
)


In [312]:
column_mapping

ColumnMapping(target='target', prediction='prediction', datetime='datetime', id=None, numerical_features=None, categorical_features=None, datetime_features=None, target_names=None, task=None, pos_label=1, text_features=None, embeddings={'embeddings_z': ['z_0', 'z_1', 'z_2', 'z_3', 'z_4', 'z_5', 'z_6', 'z_7', 'z_8', 'z_9', 'z_10', 'z_11', 'z_12', 'z_13', 'z_14', 'z_15', 'z_16', 'z_17', 'z_18', 'z_19', 'z_20', 'z_21', 'z_22', 'z_23', 'z_24', 'z_25', 'z_26', 'z_27', 'z_28', 'z_29', 'z_30', 'z_31', 'z_32', 'z_33', 'z_34', 'z_35', 'z_36', 'z_37', 'z_38', 'z_39', 'z_40', 'z_41', 'z_42', 'z_43', 'z_44', 'z_45', 'z_46', 'z_47', 'z_48', 'z_49', 'z_50', 'z_51', 'z_52', 'z_53', 'z_54', 'z_55', 'z_56', 'z_57', 'z_58', 'z_59', 'z_60', 'z_61', 'z_62', 'z_63', 'z_64', 'z_65', 'z_66', 'z_67', 'z_68', 'z_69', 'z_70', 'z_71', 'z_72', 'z_73', 'z_74', 'z_75', 'z_76', 'z_77', 'z_78', 'z_79', 'z_80', 'z_81', 'z_82', 'z_83', 'z_84', 'z_85', 'z_86', 'z_87', 'z_88', 'z_89', 'z_90', 'z_91', 'z_92', 'z_93', 'z_9

In [313]:
embeddings_drift_metric_mmd = EmbeddingsDriftMetric(
    'embeddings_z',
    drift_method = mmd(
                            threshold = 0.015,
                            bootstrap = None,
                            quantile_probability = 0.95,
                            pca_components = None,
                        )
)

embeddings_drift_metric_ratio = EmbeddingsDriftMetric(
    'embeddings_minmax',
    drift_method = ratio(
                            component_stattest = 'wasserstein',
                            component_stattest_threshold = 0.1,
                            threshold = 0.2,
                            pca_components = None,
                        )
) 

embeddings_drift_metric_model = EmbeddingsDriftMetric(
    'embeddings_l2',
    drift_method = model(
                            threshold = 0.55,
                            bootstrap = None,
                            quantile_probability = 0.95,
                            pca_components = None,
                        )
) 

""" embeddings_drift_metric_distance = EmbeddingsDriftMetric(
    'embeddings_l2',
    drift_method = distance(
                            dist = 'euclidean', #"euclidean", "cosine", "cityblock" or "chebyshev"
                            threshold = 0.2,
                            pca_components = None,
                            bootstrap = None,
                            quantile_probability = 0.95
                        )
) """

' embeddings_drift_metric_distance = EmbeddingsDriftMetric(\n    \'embeddings_l2\',\n    drift_method = distance(\n                            dist = \'euclidean\', #"euclidean", "cosine", "cityblock" or "chebyshev"\n                            threshold = 0.2,\n                            pca_components = None,\n                            bootstrap = None,\n                            quantile_probability = 0.95\n                        )\n) '

In [314]:
model_description = f"""
#  Drift detection between {reference_split_name} and {current_split_name}. Normalized
"""


In [315]:
print(embeddings_train_z_df.var())


z_0      1.000010
z_1      1.000017
z_2      1.000019
z_3      1.000013
z_4      1.000016
           ...   
z_345    1.000015
z_346    1.000017
z_347    1.000014
z_348    1.000010
z_349    1.000027
Length: 350, dtype: float32


In [316]:
# Create the report and add the metric
report = Report(metrics=[
    Comment(model_description),
    embeddings_drift_metric_mmd,
    embeddings_drift_metric_ratio,
    embeddings_drift_metric_model
    #embeddings_drift_metric_distance
])

In [317]:
# Run the report to calculate the drift
report.run(reference_data=embeddings_train_combined_df, current_data=embeddings_test_combined_df, column_mapping=column_mapping)

In [318]:
# Save the report as HTML
path = f"{full_path}{file_name}"
report.save_html(path)

In [319]:
# Extract the results as a dictionary
report_json = report.as_dict()

In [320]:
report_json

{'metrics': [{'metric': 'Comment', 'result': {}},
  {'metric': 'EmbeddingsDriftMetric',
   'result': {'embeddings_name': 'embeddings_z',
    'drift_score': 0.003930340277983335,
    'drift_detected': False,
    'method_name': 'mmd'}},
  {'metric': 'EmbeddingsDriftMetric',
   'result': {'embeddings_name': 'embeddings_minmax',
    'drift_score': 0.7514285714285714,
    'drift_detected': True,
    'method_name': 'ratio'}},
  {'metric': 'EmbeddingsDriftMetric',
   'result': {'embeddings_name': 'embeddings_l2',
    'drift_score': 0.5472616611614446,
    'drift_detected': False,
    'method_name': 'model'}}]}

In [321]:
# Check if drift was detected
mmd_result = report_json['metrics'][1]['result']['drift_detected']
ratio_result = report_json['metrics'][2]['result']['drift_detected']
model_result = report_json['metrics'][3]['result']['drift_detected']
#distance_result = report_json['metrics'][4]['result']['drift_detected']



In [322]:
run_name_mlf= f"Drift detection {reference_split_name} vs {current_split_name}"

In [323]:
""" model = load_model("/Users/ximenamoure/Desktop/drift_last/models/MobileNetV2_0p5_all.h5") """

' model = load_model("/Users/ximenamoure/Desktop/drift_last/models/MobileNetV2_0p5_all.h5") '

In [324]:
# Start an MLflow run
with mlflow.start_run(run_name=run_name_mlf):

    #mlflow.keras.log_model(model, "model")

    mlflow.log_param("reference_split_name", reference_split_name)
    mlflow.log_param("current_split_name", current_split_name)

    # Log whether drift was detected (True/False)
    mlflow.log_param("mmd_drift_detected", mmd_result)
    mlflow.log_param("ratio_drift_detected", ratio_result)
    mlflow.log_param("model_drift_detected", model_result)
   #mlflow.log_param("distance_drift_detected", distance_result)

    # Log the drift scores if available
    mlflow.log_metric("mmd_drift_score", report_json['metrics'][1]['result']['drift_score'])
    mlflow.log_metric("ratio_drift_score", report_json['metrics'][2]['result']['drift_score'])
    mlflow.log_metric("model_drift_score", report_json['metrics'][3]['result']['drift_score'])
    #mlflow.log_metric("distance_drift_score", report_json['metrics'][4]['result']['drift_score'])

    # Log the HTML report as an artifact
    artifact_path = f"{full_path}{file_name}"
    mlflow.log_artifact(artifact_path)
    
    #path = f"{full_path}{html_path}"
    #mlflow.log_artifact(path)
    

print("Drift detection results logged in MLflow.")



Drift detection results logged in MLflow.
